In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
import sys
import time
from typing import Dict, List, Optional
from collections import Counter, defaultdict

import networkx as nx
import numpy as np
import pandas as pd
from IPython.display import clear_output
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec13.txt'

In [2]:
class BoardInputWrapper:
    def __init__(self, robot, auto: bool = False, print_rate: int = 1000, sleep_time: float = 0.5):
        self.round = 0
        self.robot = robot
        self.auto = auto
        self.print_rate = print_rate
        self.sleep_time = sleep_time
        
    def get(self):
        self.round += 1
        if self.round % self.print_rate == 0:
            self.robot.printme(clear=True)
            time.sleep(self.sleep_time)
        if self.auto:
            if self.robot.paddle_x < self.robot.ball_x:
                return 1
            elif self.robot.paddle_x > self.robot.ball_x:
                return -1
            else:
                return 0
        else:
            val = int(input('Left (-1) Stay (0) Right (1): '))
        return val

    def __bool__(self):
        return True

BLOCK_VAL = 2
PADDLE_VAL = 3
BALL_VAL = 4

class BoardOutputWrapper:
    def __init__(self, robot):
        self.robot = robot
        self.parity = 0
        self.pos_x = None
        self.pos_y = None
    
    def __call__(self, val):
        if self.parity == 0:
          self.pos_x = val
          self.parity = 1
        elif self.parity == 1:
          self.pos_y = val
          self.parity = 2
        elif self.parity == 2:
          if self.pos_x == -1 and self.pos_y == 0:
            self.robot.score = val
          else:
            pos = (self.pos_x, self.pos_y)
            self.robot.painted[pos] = val
            if val == BALL_VAL:
              self.robot.ball_x = self.pos_x
            elif val == PADDLE_VAL:
              self.robot.paddle_x = self.pos_x
          self.parity = 0
        return False

class Robot:
    def __init__(self, ops):
        self.ops = ops
        self.score = 0
        self.painted = defaultdict(int)
        self.ball_x = None
        self.paddle_x = None
    
    def simulate(self, auto: bool = False):
        output_wrapper = BoardOutputWrapper(self)
        input_wrapper = BoardInputWrapper(self, auto=auto)
        sim.simulate(self.ops, inputs=input_wrapper, output_func=output_wrapper)
        self.printme(clear=True)
    
    def printme(self, clear: bool = False):
        if clear:
            clear_output()
            
        min_x = min(key[0] for key in self.painted.keys())
        max_x = max(key[0] for key in self.painted.keys())
        min_y = min(key[1] for key in self.painted.keys())
        max_y = max(key[1] for key in self.painted.keys())
        for y in range(min_y, max_y + 1):
            for x in range(min_x + max_x + 1):
                val = self.painted[(x, y)]
                if val == 0:
                    print('.', end='')
                elif val == 1:
                    print('#', end='')
                elif val == BLOCK_VAL:
                    print('B', end='')
                elif val == PADDLE_VAL:
                    print('_', end='')
                elif val == BALL_VAL:
                    print('O', end='')
                else:
                    raise ValueError('Invalid return value')
                print('.' if self.painted[(x, y)] == 0 else '#', end='')
            print()
        print(f'Score: {self.score}')
        sys.stdout.flush()

In [3]:
robot = Robot(sim.read_ops(INPUT_PATH.read_text().strip()))
robot.simulate()

##################################################################################
##..............................................................................##
##....B#B#....B#B#B#..B#B#..B#B#..B#..B#B#B#B#......B#B#B#B#..B#B#B#B#..B#B#B#..##
##..B#B#......B#B#B#....B#B#B#..........B#B#B#B#B#..B#..B#B#B#B#B#B#....B#B#....##
##..B#B#B#B#B#B#B#..B#B#B#B#B#..B#B#....B#B#B#..B#B#B#....B#....B#B#B#B#B#..B#..##
##..B#B#B#B#B#..B#B#B#..B#B#B#B#........B#B#B#B#B#B#..B#B#......B#B#B#......B#..##
##..B#B#B#..B#B#B#B#B#B#B#B#........B#..B#..B#B#B#..B#B#B#B#B#B#B#B#..B#..B#B#..##
##..B#B#B#B#B#B#....B#......B#B#B#....B#..B#B#B#B#B#B#B#B#B#B#B#B#B#B#....B#B#..##
##....B#B#B#B#..B#B#B#..B#B#B#B#B#B#B#B#..B#B#B#B#B#....B#B#B#B#B#......B#..B#..##
##....B#B#..B#B#..B#B#....B#B#B#B#B#..B#B#..B#B#B#B#B#B#B#B#B#B#B#..B#..B#B#B#..##
##..B#..B#....B#....B#B#B#..B#B#..B#B#B#B#B#B#B#B#..B#B#B#B#B#..B#B#B#B#B#B#....##
##..B#B#B#B#B#B#B#B#B#..B#B#..B#B#......B#B#..B#B#B#B#B#B#B#..B#B#....B#..B#....##
##..

In [4]:
sum(val == 2 for val in robot.painted.values())

420

In [5]:
robot.printme()

##################################################################################
##..............................................................................##
##....B#B#....B#B#B#..B#B#..B#B#..B#..B#B#B#B#......B#B#B#B#..B#B#B#B#..B#B#B#..##
##..B#B#......B#B#B#....B#B#B#..........B#B#B#B#B#..B#..B#B#B#B#B#B#....B#B#....##
##..B#B#B#B#B#B#B#..B#B#B#B#B#..B#B#....B#B#B#..B#B#B#....B#....B#B#B#B#B#..B#..##
##..B#B#B#B#B#..B#B#B#..B#B#B#B#........B#B#B#B#B#B#..B#B#......B#B#B#......B#..##
##..B#B#B#..B#B#B#B#B#B#B#B#........B#..B#..B#B#B#..B#B#B#B#B#B#B#B#..B#..B#B#..##
##..B#B#B#B#B#B#....B#......B#B#B#....B#..B#B#B#B#B#B#B#B#B#B#B#B#B#B#....B#B#..##
##....B#B#B#B#..B#B#B#..B#B#B#B#B#B#B#B#..B#B#B#B#B#....B#B#B#B#B#......B#..B#..##
##....B#B#..B#B#..B#B#....B#B#B#B#B#..B#B#..B#B#B#B#B#B#B#B#B#B#B#..B#..B#B#B#..##
##..B#..B#....B#....B#B#B#..B#B#..B#B#B#B#B#B#B#B#..B#B#B#B#B#..B#B#B#B#B#B#....##
##..B#B#B#B#B#B#B#B#B#..B#B#..B#B#......B#B#..B#B#B#B#B#B#B#..B#B#....B#..B#....##
##..

In [6]:
ops = sim.read_ops(INPUT_PATH.read_text().strip())
ops[0] = 2
robot = Robot(ops)
robot.simulate(auto=True)

##################################################################################
##..............................................................................##
##..............................................................................##
##..............................................................................##
##..............................................................................##
##..............................................................................##
##..............................................................................##
##..............................................................................##
##..............................................................................##
##..............................................................................##
##..............................................................................##
##..........................................O#..................................##
##..